In [ ]:
!pip3 install streamlit
!pip install streamlit joblib seaborn pyngrok

In [13]:
%%bash
ls -lrt

total 8
drwxr-xr-x 1 root root 4096 May  5 13:40 sample_data
drwx------ 6 root root 4096 May  6 16:35 drive


In [14]:
%%bash
cat << 'EOF' > /content/drive/MyDrive/streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

st.set_page_config(page_title="Crypto Liquidity Predictor", layout="wide")

@st.cache_data
def load_data():
    df = pd.read_csv(
        '/content/drive/MyDrive/pwskills-DataScience_and_AI/Machine Learning Project - CryptoCurrency Liquidity/data/coin_gecko_2022-03-17.csv',
        parse_dates=['date']
    )
    df.dropna(how='all', inplace=True)
    for col in ['1h','24h','7d']:
        df[col] = (
            df[col]
              .astype(str)
              .str.replace('%','')
              .str.replace('—','')
              .replace('', np.nan)
              .astype(float)
        )
    df.rename(columns={
        '1h':'pct_change_1h','24h':'pct_change_24h','7d':'pct_change_7d',
        '24h_volume':'volume_24h','mkt_cap':'market_cap'
    }, inplace=True)
    df.sort_values('market_cap', ascending=False, inplace=True)
    df['price_to_marketcap']  = df['price'] / df['market_cap']
    df['volume_to_marketcap'] = df['volume_24h'] / df['market_cap']
    df['volatility_score']    = df['pct_change_7d'].abs()
    return df

@st.cache_resource
def load_model_and_scaler():
    model  = joblib.load('/content/drive/MyDrive/pwskills-DataScience_and_AI/Machine Learning Project - CryptoCurrency Liquidity/models/best_crypto_liquidity_model.pkl')
    scaler = joblib.load('/content/drive/MyDrive/pwskills-DataScience_and_AI/Machine Learning Project - CryptoCurrency Liquidity/models/scaler.pkl')
    return model, scaler

df = load_data()
model, scaler = load_model_and_scaler()

st.title("🚀 Cryptocurrency Liquidity Predictor")

mode = st.sidebar.radio("Mode", ["EDA", "Predict"])

if mode == "EDA":
    st.subheader("Top 10 by Market Cap")
    st.bar_chart(df.nlargest(10,'market_cap')[['coin','market_cap']].set_index('coin'))

    st.subheader("Top 10 by 24h Volume")
    st.bar_chart(df.nlargest(10,'volume_24h')[['coin','volume_24h']].set_index('coin'))

    st.subheader("Feature Correlation")
    corr = df[['price','pct_change_1h','pct_change_24h','pct_change_7d',
               'volume_24h','market_cap','price_to_marketcap',
               'volume_to_marketcap','volatility_score']].corr()
    fig, ax = plt.subplots(figsize=(6,5))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", ax=ax)
    st.pyplot(fig)

else:
    st.subheader("Enter Feature Values")
    price            = st.number_input("Price (USD)", value=float(df['price'].mean()))
    pct_1h           = st.number_input("1h % Change", value=float(df['pct_change_1h'].mean()))
    pct_24h          = st.number_input("24h % Change", value=float(df['pct_change_24h'].mean()))
    pct_7d           = st.number_input("7d % Change", value=float(df['pct_change_7d'].mean()))
    vol_24h          = st.number_input("24h Volume", value=float(df['volume_24h'].mean()))
    mcap             = st.number_input("Market Cap", value=float(df['market_cap'].mean()))
    price2mcap       = st.number_input("Price / Market Cap", value=float(df['price_to_marketcap'].mean()))
    vol2mcap         = st.number_input("Volume24h / Market Cap", value=float(df['volume_to_marketcap'].mean()))
    volatility_score = st.number_input("Volatility Score (|7d %|)", value=float(df['volatility_score'].mean()))

    if st.button("Predict"):
        X_user = np.array([[price, pct_1h, pct_24h, pct_7d,
                             vol_24h, mcap, price2mcap,
                             vol2mcap, volatility_score]])
        X_scaled = scaler.transform(X_user)
        pred = model.predict(X_scaled)[0]
        st.success(f"Predicted Liquidity (Volume): {pred:,.0f}")
EOF


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from pyngrok import ngrok

!ngrok authtoken xxxxxx # Replace <your_authtoken> with your actual authtoken

# Kill any existing tunnels
ngrok.kill()

# Launch Streamlit (headless) in background
get_ipython().system_raw(
    'streamlit run /content/drive/MyDrive/streamlit_app.py '
    '--server.port 8501 '
    '--server.headless true &'
)

# Create ngrok tunnel
public_url = ngrok.connect(8501)
print(" 🌐 Streamlit app is live at:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
 🌐 Streamlit app is live at: NgrokTunnel: "https://e62f-34-91-137-46.ngrok-free.app" -> "http://localhost:8501"


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [17]:
# !pip install ydata-profiling
# # Install ydata-profiling if not already available (in local Colab/Streamlit setup, you'd run: pip install ydata-profiling)
# from ydata_profiling import ProfileReport

# # Load the dataset
# df = pd.read_csv("/content/drive/MyDrive/pwskills-DataScience_and_AI/Machine Learning Project - CryptoCurrency Liquidity/data/coin_gecko_2022-03-17.csv")

# # Generate the profiling report
# profile = ProfileReport(df, title="Cryptocurrency EDA Report", explorative=True)
# report_path = "/content/drive/MyDrive/pwskills-DataScience_and_AI/Machine Learning Project - CryptoCurrency Liquidity/docs/crypto_eda_profile_report.html"
# profile.to_file(report_path)

# report_path


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.1/400.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 687.8/687.8 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 70.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27081 sha256=eda82128f7714ae3ab754df20954fd13114cd47706f69a08f522ce8ab0648d8e
  Stored in directory: /root/.cache/pip/wheels/8d/55/1a/19cd535375ed1ede0c996405ebffe34b196d78e2d9545723a2
Successfully built htmlmin


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████| 9/9 [00:00<00:00, 47.80it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

'/content/drive/MyDrive/pwskills-DataScience_and_AI/Machine Learning Project - CryptoCurrency Liquidity/docs/crypto_eda_profile_report.html'